
<div style="direction: rtl; text-align: right; font-family: Arial, sans-serif; line-height: 1.5;">
<div style="direction: rtl; text-align: right; font-family: Arial, sans-serif; line-height: 1.5;">


<style>
  h1 { font-weight: 700; font-size: 28px; margin-bottom: 10px; }
  h2 { font-weight: 600; font-size: 22px; margin-bottom: 8px; }
  ul, ol {
    direction: rtl;
    text-align: right;
    padding-right: 20px;
    margin-top: 0; margin-bottom: 10px;
    list-style-position: inside;
  }
</style>
# מטלה להגשה בלימדת מכונה רבקה אלחג׳ ועמית אלגזר
</div>
</div>

<div style="direction: rtl; text-align: right; font-family: Arial, sans-serif; line-height: 1.5;">
<div style="direction: rtl; text-align: right; font-family: Arial, sans-serif; line-height: 1.5;">


<style>
  h1 { font-weight: 700; font-size: 28px; margin-bottom: 10px; }
  h2 { font-weight: 600; font-size: 22px; margin-bottom: 8px; }
  ul, ol {
    direction: rtl;
    text-align: right;
    padding-right: 20px;
    margin-top: 0; margin-bottom: 10px;
    list-style-position: inside;
  }
</style>


# 1. ייבוא ספריות והגדרת פונקציית הכנת הנתונים

##### בבלוק זה מייבאים את כל הספריות הנדרשות ומגדירים פונקציה שמנקה ומכינה את הנתונים:  
- המרת עמודות למספרים  
- סינון ערכים חריגים  
- יצירת פיצ'רים כמו מחיר לחדר, לוגריתם מחיר ועוד  
- טיפול בערכים חסרים בטקסט
</div>
</div>

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, KFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import randint

def prepare_data(df, mode='train'):
    """
    פונקציה לניקוי והכנת דאטה:
    - ממירה עמודות מספריות לערכים נומריים
    - מסננת ערכים חריגים לפי כללים
    - מוסיפה פיצ'רים חדשים לשיפור המודל
    - ממירה עמודות טקסט למחרוזות וממלאת ערכים חסרים
    """
    df = df.copy()  # עבודה על עותק למניעת שינוי מקורי

    # עמודות מספריות שנטפל בהן
    numeric_cols = ['area', 'room_num', 'floor', 'total_floors', 'monthly_arnona', 'price']  

    # המרת עמודות למספרים, טעויות יומרו ל-NaN
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # הסרת שורות עם ערכים חסרים בעמודות חשובות
    df.dropna(subset=['area', 'room_num', 'price'], inplace=True)

    # סינון ערכים חריגים - שטח מעל 10, מחיר מעל 1000, חדרים מעל 0
    df = df[(df['area'] > 10) & (df['price'] > 1000) & (df['room_num'] > 0)]

    # סינון מחירים חריגים גבוהים מעל 30,000
    df = df[df['price'] <= 30_000]

    # יצירת פיצ'ר מחיר פר חדר
    df['price_per_room'] = df['price'] / df['room_num']

    # לוגריתם של מחיר (לשיפור הלמידה)
    df['log_price'] = np.log1p(df['price'])

    # לוגריתם של מחיר פר חדר
    df['log_price_per_room'] = np.log1p(df['price_per_room'])

    # יחס קומה לקומות כולל טיפול בערכים חסרים
    df['floor_area_ratio'] = 0
    if ('floor' in df.columns) and ('total_floors' in df.columns):
        df['floor'] = pd.to_numeric(df['floor'], errors='coerce').fillna(0)
        # להימנע מחילוק באפס, מוחלף ל-1
        df['total_floors'] = pd.to_numeric(df['total_floors'], errors='coerce').replace(0, 1)
        df['floor_area_ratio'] = df['floor'] / df['total_floors']

    # המרת עמודות קטגוריאליות למחרוזות ומילוי ערכים חסרים ב-'nan'
    cat_cols = df.select_dtypes(include=['object']).columns.tolist()
    for col in cat_cols:
        df[col] = df[col].astype(str).fillna('nan')

    return df

<div style="direction: rtl; text-align: right; font-family: Arial, sans-serif; line-height: 1.5;">
<div style="direction: rtl; text-align: right; font-family: Arial, sans-serif; line-height: 1.5;">


<style>
  h1 { font-weight: 700; font-size: 28px; margin-bottom: 10px; }
  h2 { font-weight: 600; font-size: 22px; margin-bottom: 8px; }
  ul, ol {
    direction: rtl;
    text-align: right;
    padding-right: 20px;
    margin-top: 0; margin-bottom: 10px;
    list-style-position: inside;
  }
</style>


# 2. טעינת הקבצים והדפסת גודל הדאטה לפני ואחרי ניקוי

##### כאן נטען את הקבצים ונציג כמה שורות יש לפני ואחרי ניקוי באמצעות הפונקציה שהגדרנו.
</div>
</div>

In [4]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

print(f"[train] מספר שורות לפני ניקוי וסינון: {len(train_df)}")
print(f"[test] מספר שורות לפני ניקוי וסינון: {len(test_df)}")

# ניקוי והכנת דאטה בעזרת הפונקציה
train_df = prepare_data(train_df, mode='train')
test_df = prepare_data(test_df, mode='test')

print(f"[train] מספר שורות אחרי ניקוי וסינון: {len(train_df)}")
print(f"[test] מספר שורות אחרי ניקוי וסינון: {len(test_df)}")

[train] מספר שורות לפני ניקוי וסינון: 788
[test] מספר שורות לפני ניקוי וסינון: 79
[train] מספר שורות אחרי ניקוי וסינון: 715
[test] מספר שורות אחרי ניקוי וסינון: 74


<div style="direction: rtl; text-align: right; font-family: Arial, sans-serif; line-height: 1.5;">
<div style="direction: rtl; text-align: right; font-family: Arial, sans-serif; line-height: 1.5;">


<style>
  h1 { font-weight: 700; font-size: 28px; margin-bottom: 10px; }
  h2 { font-weight: 600; font-size: 22px; margin-bottom: 8px; }
  ul, ol {
    direction: rtl;
    text-align: right;
    padding-right: 20px;
    margin-top: 0; margin-bottom: 10px;
    list-style-position: inside;
  }
</style>


# 3. הגדרת משתני למידה: מטרות (target) ותכונות (features)
##### מחלקים את הנתונים למטרות ולפיצ'רים, ומגדירים אילו עמודות מספריות ואילו קטגוריאליות לעיבוד שונה.
</div>
</div>

In [6]:
# העמודה שאנו רוצים לחזות היא 'log_price'
target = 'log_price'

# פיצ'רים - מורידים את מחיר הגולמי והלוגריתם שלו
X_train = train_df.drop(columns=['price', 'log_price'])
y_train = train_df[target]

# בנתוני בדיקה, מחיר יכול להיות חסר ולכן נבדוק
X_test = test_df.drop(columns=['price'], errors='ignore')
y_test = np.log1p(test_df['price']) if 'price' in test_df.columns else None

# זיהוי עמודות נומריות וקטגוריאליות לפיצול עיבוד
numeric_features = X_train.select_dtypes(include=['number']).columns.tolist()
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()

<div style="direction: rtl; text-align: right; font-family: Arial, sans-serif; line-height: 1.5;">
<div style="direction: rtl; text-align: right; font-family: Arial, sans-serif; line-height: 1.5;">


<style>
  h1 { font-weight: 700; font-size: 28px; margin-bottom: 10px; }
  h2 { font-weight: 600; font-size: 22px; margin-bottom: 8px; }
  ul, ol {
    direction: rtl;
    text-align: right;
    padding-right: 20px;
    margin-top: 0; margin-bottom: 10px;
    list-style-position: inside;
  }
</style>


# 4. בניית Pipeline לעיבוד נומרי וקטגוריאלי

##### נגדיר טיפול נומרי שכולל מילוי ערכים חסרים וסקלינג, וטיפול בקטגוריות עם המרה לאנקום-אנקודינג.
</div>
</div>

In [8]:
# טיפול נומרי: מילוי חוסרים במדד מדיאן + סקלינג סטנדרטי
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# טיפול בקטגוריות: מילוי בערך קבוע + one-hot encoding עם התעלמות מערכים לא מוכרים
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# שילוב שני הטיפולים בעמודות מתאימות
preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

<div style="direction: rtl; text-align: right; font-family: Arial, sans-serif; line-height: 1.5;">
<div style="direction: rtl; text-align: right; font-family: Arial, sans-serif; line-height: 1.5;">


<style>
  h1 { font-weight: 700; font-size: 28px; margin-bottom: 10px; }
  h2 { font-weight: 600; font-size: 22px; margin-bottom: 8px; }
  ul, ol {
    direction: rtl;
    text-align: right;
    padding-right: 20px;
    margin-top: 0; margin-bottom: 10px;
    list-style-position: inside;
  }
</style>


# 5. ElasticNet - הגדרת Pipeline, פרמטרים ו-Grid Search

##### נגדיר מודל ElasticNet ונבצע חיפוש פרמטרים אופטימלי על ידי Cross-Validation עם 10 קפלים.
</div>
</div>

In [10]:
pipe_en = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', ElasticNet(max_iter=5000))
])

# טווח פרמטרים לחיפוש
param_grid_en = {
    'regressor__alpha': np.logspace(-3, 1, 20),  # אלפא בין 0.001 ל-10 בלוגריתם
    'regressor__l1_ratio': np.linspace(0.1, 0.9, 9)  # יחס בין L1 ל-L2 רגולריזציה
}

cv = KFold(n_splits=10, shuffle=True, random_state=42)  # הגדרת Cross-Validation

print("מתחילים אימון ElasticNet GridSearchCV...")
grid_search_en = GridSearchCV(pipe_en, param_grid_en, cv=cv, scoring='neg_mean_squared_error', n_jobs=-1, verbose=0)
grid_search_en.fit(X_train, y_train)

best_en = grid_search_en.best_estimator_  # המודל הטוב ביותר

print(f"ElasticNet Best params: {grid_search_en.best_params_}")

מתחילים אימון ElasticNet GridSearchCV...
ElasticNet Best params: {'regressor__alpha': 0.001, 'regressor__l1_ratio': 0.2}


<div style="direction: rtl; text-align: right; font-family: Arial, sans-serif; line-height: 1.5;">
<div style="direction: rtl; text-align: right; font-family: Arial, sans-serif; line-height: 1.5;">


<style>
  h1 { font-weight: 700; font-size: 28px; margin-bottom: 10px; }
  h2 { font-weight: 600; font-size: 22px; margin-bottom: 8px; }
  ul, ol {
    direction: rtl;
    text-align: right;
    padding-right: 20px;
    margin-top: 0; margin-bottom: 10px;
    list-style-position: inside;
  }
</style>


# 6. Decision Tree עם Grid Search

##### נגדיר מודל עץ החלטה ונבצע GridSearch על פרמטרים נפוצים.
</div>
</div>

In [12]:
pipe_dt = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', DecisionTreeRegressor(random_state=42))
])

param_grid_dt = {
    'regressor__max_depth': [5, 10, 20, 30, None],  # עומק העץ
    'regressor__min_samples_split': [2, 5, 10],  # מינימום דגימות לפיצול
    'regressor__min_samples_leaf': [1, 2, 4]  # מינימום דגימות בעלה
}

print("מתחילים אימון Decision Tree GridSearchCV...")
grid_search_dt = GridSearchCV(pipe_dt, param_grid_dt, cv=cv, scoring='neg_mean_squared_error', n_jobs=-1, verbose=0)
grid_search_dt.fit(X_train, y_train)

best_dt_grid = grid_search_dt.best_estimator_

print(f"Decision Tree GridSearch Best params: {grid_search_dt.best_params_}")

מתחילים אימון Decision Tree GridSearchCV...
Decision Tree GridSearch Best params: {'regressor__max_depth': 20, 'regressor__min_samples_leaf': 2, 'regressor__min_samples_split': 10}


<div style="direction: rtl; text-align: right; font-family: Arial, sans-serif; line-height: 1.5;">
<div style="direction: rtl; text-align: right; font-family: Arial, sans-serif; line-height: 1.5;">


<style>
  h1 { font-weight: 700; font-size: 28px; margin-bottom: 10px; }
  h2 { font-weight: 600; font-size: 22px; margin-bottom: 8px; }
  ul, ol {
    direction: rtl;
    text-align: right;
    padding-right: 20px;
    margin-top: 0; margin-bottom: 10px;
    list-style-position: inside;
  }
</style>


# 7. Decision Tree עם Randomized Search

##### נבצע חיפוש אקראי על פרמטרים דומים עם מספר איטרציות מוגבל.
</div>
</div>

In [14]:
param_dist_dt = {
    'regressor__max_depth': [5, 10, 20, 30, None],
    'regressor__min_samples_split': randint(2, 20),
    'regressor__min_samples_leaf': randint(1, 10)
}

print("מתחילים אימון Decision Tree RandomizedSearchCV...")
random_search_dt = RandomizedSearchCV(pipe_dt, param_distributions=param_dist_dt, n_iter=50, cv=cv, scoring='neg_mean_squared_error', n_jobs=-1, random_state=42, verbose=0)
random_search_dt.fit(X_train, y_train)

best_dt_rand = random_search_dt.best_estimator_

print(f"Decision Tree RandomizedSearch Best params: {random_search_dt.best_params_}")

מתחילים אימון Decision Tree RandomizedSearchCV...
Decision Tree RandomizedSearch Best params: {'regressor__max_depth': 10, 'regressor__min_samples_leaf': 2, 'regressor__min_samples_split': 11}


<div style="direction: rtl; text-align: right; font-family: Arial, sans-serif; line-height: 1.5;">
<div style="direction: rtl; text-align: right; font-family: Arial, sans-serif; line-height: 1.5;">


<style>
  h1 { font-weight: 700; font-size: 28px; margin-bottom: 10px; }
  h2 { font-weight: 600; font-size: 22px; margin-bottom: 8px; }
  ul, ol {
    direction: rtl;
    text-align: right;
    padding-right: 20px;
    margin-top: 0; margin-bottom: 10px;
    list-style-position: inside;
  }
</style>


# 8. פונקציה לחיזוי והערכת ביצועים

##### הפונקציה מחזירה מדדי RMSE ו-R2 בשפה של מחירי אמת (לא בלוגריתם).
</div>
</div>

In [16]:
def predict_and_eval(model, X_train, y_train, X_test, y_test):
    # חיזוי בלוגריתם על אימון ובדיקה
    y_train_pred_log = model.predict(X_train)
    y_test_pred_log = model.predict(X_test) if X_test is not None else None

    # המרת מטרות בלוגריתם למחיר אמיתי
    y_train_true = np.expm1(y_train)
    y_test_true = np.expm1(y_test) if y_test is not None else None

    # המרת חיזוי בלוגריתם למחיר אמיתי
    y_train_pred = np.expm1(y_train_pred_log)
    y_test_pred = np.expm1(y_test_pred_log) if y_test_pred_log is not None else None

    # חישוב מדדים RMSE ו-R2
    def calc_metrics(y_true, y_pred):
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        r2 = r2_score(y_true, y_pred)
        return rmse, r2

    train_rmse, train_r2 = calc_metrics(y_train_true, y_train_pred)
    test_rmse, test_r2 = calc_metrics(y_test_true, y_test_pred) if y_test_true is not None else (None, None)

    return (train_rmse, train_r2), (test_rmse, test_r2), y_test_pred

<div style="direction: rtl; text-align: right; font-family: Arial, sans-serif; line-height: 1.5;">
<div style="direction: rtl; text-align: right; font-family: Arial, sans-serif; line-height: 1.5;">


<style>
  h1 { font-weight: 700; font-size: 28px; margin-bottom: 10px; }
  h2 { font-weight: 600; font-size: 22px; margin-bottom: 8px; }
  ul, ol {
    direction: rtl;
    text-align: right;
    padding-right: 20px;
    margin-top: 0; margin-bottom: 10px;
    list-style-position: inside;
  }
</style>


# 9. הערכת ביצועים והצגת התוצאות לכל המודלים
</div>
</div>

In [18]:
(train_rmse_en, train_r2_en), (test_rmse_en, test_r2_en), y_test_pred_en = predict_and_eval(best_en, X_train, y_train, X_test, y_test)
(train_rmse_dtg, train_r2_dtg), (test_rmse_dtg, test_r2_dtg), y_test_pred_dtg = predict_and_eval(best_dt_grid, X_train, y_train, X_test, y_test)
(train_rmse_dtr, train_r2_dtr), (test_rmse_dtr, test_r2_dtr), y_test_pred_dtr = predict_and_eval(best_dt_rand, X_train, y_train, X_test, y_test)

print("\n=== תוצאות ElasticNet ===")
print(f"Train RMSE: {train_rmse_en:.2f}, R2: {train_r2_en:.4f}")
print(f"Test RMSE: {test_rmse_en:.2f}, R2: {test_r2_en:.4f}")

print("\n=== תוצאות Decision Tree Grid Search ===")
print(f"Train RMSE: {train_rmse_dtg:.2f}, R2: {train_r2_dtg:.4f}")
print(f"Test RMSE: {test_rmse_dtg:.2f}, R2: {test_r2_dtg:.4f}")

print("\n=== תוצאות Decision Tree Randomized Search ===")
print(f"Train RMSE: {train_rmse_dtr:.2f}, R2: {train_r2_dtr:.4f}")
print(f"Test RMSE: {test_rmse_dtr:.2f}, R2: {test_r2_dtr:.4f}")


=== תוצאות ElasticNet ===
Train RMSE: 578.43, R2: 0.9806
Test RMSE: 475.86, R2: 0.9779

=== תוצאות Decision Tree Grid Search ===
Train RMSE: 479.65, R2: 0.9866
Test RMSE: 391.76, R2: 0.9850

=== תוצאות Decision Tree Randomized Search ===
Train RMSE: 583.22, R2: 0.9802
Test RMSE: 403.28, R2: 0.9842


<div style="direction: rtl; text-align: right; font-family: Arial, sans-serif; line-height: 1.5;">
<div style="direction: rtl; text-align: right; font-family: Arial, sans-serif; line-height: 1.5;">


<style>
  h1 { font-weight: 700; font-size: 28px; margin-bottom: 10px; }
  h2 { font-weight: 600; font-size: 22px; margin-bottom: 8px; }
  ul, ol {
    direction: rtl;
    text-align: right;
    padding-right: 20px;
    margin-top: 0; margin-bottom: 10px;
    list-style-position: inside;
  }
</style>


# 10. פונקציה לשליפת שמות פיצ'רים מתוך preprocessor מאומן
##### חשוב לשלוף את כל שמות הפיצ׳רים אחרי עיבוד כדי להבין מהם את הפיצ׳רים החשובים במודל.
</div>
</div>

In [20]:
def get_feature_names(preprocessor_fitted):
    # שמות הפיצ'רים הנומריים נשארים כמו שהם
    num_features = numeric_features

    # שמות הפיצ'רים הקטגוריאליים אחרי one-hot encoding
    cat_features = list(preprocessor_fitted.named_transformers_['cat']
                        .named_steps['onehot']
                        .get_feature_names_out(categorical_features))
    return num_features + cat_features

feature_names_en = get_feature_names(best_en.named_steps['preprocessor'])
feature_names_dtg = get_feature_names(best_dt_grid.named_steps['preprocessor'])
feature_names_dtr = get_feature_names(best_dt_rand.named_steps['preprocessor'])

<div style="direction: rtl; text-align: right; font-family: Arial, sans-serif; line-height: 1.5;">
<div style="direction: rtl; text-align: right; font-family: Arial, sans-serif; line-height: 1.5;">


<style>
  h1 { font-weight: 700; font-size: 28px; margin-bottom: 10px; }
  h2 { font-weight: 600; font-size: 22px; margin-bottom: 8px; }
  ul, ol {
    direction: rtl;
    text-align: right;
    padding-right: 20px;
    margin-top: 0; margin-bottom: 10px;
    list-style-position: inside;
  }
</style>


# 11. הצגת 5 הפיצ'רים החשובים ביותר לפי מודל ElasticNet
</div>
</div>

In [22]:
coefs = best_en.named_steps['regressor'].coef_
coefs_df = pd.DataFrame({'feature': feature_names_en, 'coefficient': coefs})

# ערך מוחלט של המקדמים לצורך דירוג
coefs_df['abs_coef'] = coefs_df['coefficient'].abs()

# בחירת חמשת הפיצ'רים עם המקדמים הכי גדולים
top5_en = coefs_df.sort_values('abs_coef', ascending=False).head(5)

print("\nחמשת התכונות עם המקדמים הכי גדולים ב-ElasticNet:")
print(top5_en[['feature', 'coefficient']])


חמשת התכונות עם המקדמים הכי גדולים ב-ElasticNet:
                                                feature  coefficient
22                                   log_price_per_room     0.400101
0                                              room_num     0.356807
40                            property_type_סטודיו/לופט    -0.177234
1205  description_נחלת בנימין42 תלאביב. דירה מרוהטת ...    -0.118685
535                              address_נחלת בנימין 42    -0.118683


<div style="direction: rtl; text-align: right; font-family: Arial, sans-serif; line-height: 1.5;">
<div style="direction: rtl; text-align: right; font-family: Arial, sans-serif; line-height: 1.5;">


<style>
  h1 { font-weight: 700; font-size: 28px; margin-bottom: 10px; }
  h2 { font-weight: 600; font-size: 22px; margin-bottom: 8px; }
  ul, ol {
    direction: rtl;
    text-align: right;
    padding-right: 20px;
    margin-top: 0; margin-bottom: 10px;
    list-style-position: inside;
  }
</style>


# 12. הצגת 5 הפיצ'רים החשובים ביותר לפי Decision Tree Grid Search
</div>
</div>

In [24]:
importances_dtg = best_dt_grid.named_steps['regressor'].feature_importances_
imp_dtg_df = pd.DataFrame({'feature': feature_names_dtg, 'importance': importances_dtg})

top5_dtg = imp_dtg_df.sort_values('importance', ascending=False).head(5)

print("\nחמשת הפיצ'רים הכי חשובים ב-Decision Tree Grid Search:")
print(top5_dtg)


חמשת הפיצ'רים הכי חשובים ב-Decision Tree Grid Search:
               feature  importance
22  log_price_per_room    0.365292
2                 area    0.349036
0             room_num    0.155761
21      price_per_room    0.125817
7         building_tax    0.000929


<div style="direction: rtl; text-align: right; font-family: Arial, sans-serif; line-height: 1.5;">
<div style="direction: rtl; text-align: right; font-family: Arial, sans-serif; line-height: 1.5;">


<style>
  h1 { font-weight: 700; font-size: 28px; margin-bottom: 10px; }
  h2 { font-weight: 600; font-size: 22px; margin-bottom: 8px; }
  ul, ol {
    direction: rtl;
    text-align: right;
    padding-right: 20px;
    margin-top: 0; margin-bottom: 10px;
    list-style-position: inside;
  }
</style>


# 13. הצגת 5 הפיצ'רים החשובים ביותר לפי Decision Tree Randomized Search
</div>
</div>

In [26]:
importances_dtr = best_dt_rand.named_steps['regressor'].feature_importances_
imp_dtr_df = pd.DataFrame({'feature': feature_names_dtr, 'importance': importances_dtr})

top5_dtr = imp_dtr_df.sort_values('importance', ascending=False).head(5)

print("\nחמשת הפיצ'רים הכי חשובים ב-Decision Tree Randomized Search:")
print(top5_dtr)


חמשת הפיצ'רים הכי חשובים ב-Decision Tree Randomized Search:
               feature  importance
21      price_per_room    0.434998
2                 area    0.349630
0             room_num    0.155743
22  log_price_per_room    0.057119
6       monthly_arnona    0.000590


<div style="direction: rtl; text-align: right; font-family: Arial, sans-serif; line-height: 1.5;">
<style>
  h1 { font-weight: 700; font-size: 28px; margin-bottom: 10px; }
  h2 { font-weight: 600; font-size: 22px; margin-bottom: 8px; }
  ul, ol {
    direction: rtl;
    text-align: right;
    padding-right: 20px;
    margin-top: 0; margin-bottom: 10px;
    list-style-position: inside;
  }
</style>

# 14. השוואה בין Decision Tree Grid Search ל- Decision Tree Randomized Search:
#### במקרה זה, Decision Tree Grid Search הניב תוצאות טובות יותר כי הוא בדק באופן יסודי ומקיף את כל צירופי הפרמטרים האפשריים בטווח שהוגדר, מה שאפשר למצוא את השילוב האופטימלי למודל הן באימון והן בבדיקה. לעומת זאת, Decision Tree Randomized Search מבצע חיפוש אקראי ומוגבל במספר האיטרציות, ולכן ייתכן שלא נבחרו הפרמטרים הטובים ביותר, מה שהביא לתוצאות מעט פחות מדויקות. לכן, כאשר טווח הפרמטרים אינו גדול מדי והזמן לא מהווה מגבלה משמעותית, Grid Search מספק חיפוש מדויק ומעמיק יותר, דבר שהשפיע על הביצועים הטובים יותר של המודל במקרה זה.

</div>


<div style="direction: rtl; text-align: right; font-family: Arial, sans-serif; line-height: 1.5;">
<div style="direction: rtl; text-align: right; font-family: Arial, sans-serif; line-height: 1.5;">


<style>
  h1 { font-weight: 700; font-size: 28px; margin-bottom: 10px; }
  h2 { font-weight: 600; font-size: 22px; margin-bottom: 8px; }
  ul, ol {
    direction: rtl;
    text-align: right;
    padding-right: 20px;
    margin-top: 0; margin-bottom: 10px;
    list-style-position: inside;
  }
</style>


# 15.השוואה בין מודל Elastic Net למודל Decision Tree Grid Search:
#### במקרה זה, ElasticNet נתן תוצאות טובות יותר מה-Decision Tree Grid Search בעיקר בגלל שהוא מודל ליניארי עם רגולריזציה שמאזן בין פשטות למורכבות, ומסוגל להתמודד היטב עם תכונות מרובות ורעשים בנתונים. לעומת זאת, עץ ההחלטה עלול להיות רגיש לאוברפיטינג ולתנודות בנתונים, במיוחד כאשר עומק העץ גבוה, מה שגורם לו להצטיין פחות על קבוצת הבדיקה. בנוסף, ElasticNet עם חיפוש פרמטרים מדויק הצליח למצוא איזון טוב בין L1 ו-L2, ששיפר את היכולת הכללית של המודל לחזות נכון את המחירים. לכן, בבחינה הכוללת של דיוק, גמישות ויכולת הכללה, ElasticNet הציג ביצועים טובים יותר.
</div>
</div>

<div style="direction: rtl; text-align: right; font-family: Arial, sans-serif; line-height: 1.5;">
<div style="direction: rtl; text-align: right; font-family: Arial, sans-serif; line-height: 1.5;">


<style>
  h1 { font-weight: 700; font-size: 28px; margin-bottom: 10px; }
  h2 { font-weight: 600; font-size: 22px; margin-bottom: 8px; }
  ul, ol {
    direction: rtl;
    text-align: right;
    padding-right: 20px;
    margin-top: 0; margin-bottom: 10px;
    list-style-position: inside;
  }
</style>


# 16. האם קיימת התאמה בין המאפיינים החשובים ביותר בכל אחד מן המודלים?
#### למרות שכל שלושת המודלים מצביעים על חשיבות גבוהה לפיצ'רים כמו מספר החדרים, שטח הדירה והמחיר ללוגריתם פר חדר, ElasticNet בולט בכך שהוא גם כולל תכונות קטגוריאליות וטקסטואליות ספציפיות (כמו סוג הנכס וכתובת) בשל אופיו הליניארי והיכולת שלו לטפל במשתנים אלו במקדמים מותאמים, בעוד שעצי ההחלטה מתמקדים בעיקר בתכונות מספריות כלליות ומדגישים יותר את המחיר לפר חדר ושטח הדירה. ההבדל הזה נובע מהאופן שבו כל מודל מעבד את המידע: ElasticNet מחפש קשרים ליניאריים ויכול להעניק משקל לפיצ'רים מפורטים, בעוד שעצי החלטה בונים מבנה היררכי של החלטות בהתאם לתכונות בולטות ופחות מושפעים מפרטים קטנים. לכן, קיימת התאמה בסיסית במאפיינים המרכזיים, אך ElasticNet מנצל טוב יותר את המידע המורכב והטקסטואלי שבנתונים.
</div>
</div>

<div style="direction: rtl; text-align: right; font-family: Arial, sans-serif; line-height: 1.5;">
<div style="direction: rtl; text-align: right; font-family: Arial, sans-serif; line-height: 1.5;">


<style>
  h1 { font-weight: 700; font-size: 28px; margin-bottom: 10px; }
  h2 { font-weight: 600; font-size: 22px; margin-bottom: 8px; }
  ul, ol {
    direction: rtl;
    text-align: right;
    padding-right: 20px;
    margin-top: 0; margin-bottom: 10px;
    list-style-position: inside;
  }
</style>


# 17. הסבירו את ההבדלים או הדימיון בין המודלים ואת הסיבות האפשריות בין הבדלים אלו
#### המודלים ElasticNet ועצי ההחלטה חולקים דמיון בכך שהם מזהים את החשיבות הגבוהה של פיצ'רים מרכזיים כמו מספר חדרים, שטח הדירה ומחיר פר חדר, אך נבדלים באופן שבו הם מתמודדים עם המידע: ElasticNet, כמודל ליניארי עם רגולריזציה, מסוגל להקצות משקלים מדויקים גם לפיצ'רים קטגוריאליים וטקסטואליים ולהפיק מהם תובנות משמעותיות, בעוד שעצי ההחלטה מתמקדים בעיקר בתכונות מספריות בולטות ובונים החלטות היררכיות שמדגישות משתנים כלליים ופחות מושפעים מפרטים קטנים. ההבדלים האלה נובעים מהמבנה האלגוריתמי השונה של המודלים – ElasticNet מחפש קשרים ליניאריים גמישים ואילו עצי ההחלטה מחפשים חלוקות ברורות בנתונים, מה שמוביל להתמקדות בפיצ'רים שונים ולדרגות חשיבות משתנות.
</div>
</div>